<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

To execute a marked cell press **Shift + Enter**

# ECOICOP Annotation
  
Use this tool to annotate your product texts with the correspondant eCoiCop category. 


## Get code from GitHub
  
At first we need to download the code from Github to here.

In [1]:
! git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'
#import sys
#sys.path.insert(1, "Thesis_Multilingual_Transferlearning")
%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import pandas as pd
import numpy as np
from tqdm import tqdm

Cloning into 'Thesis_Multilingual_Transferlearning'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 153 (delta 81), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (153/153), 26.19 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/Thesis_Multilingual_Transferlearning


## Upload CSV / Excel from Local Drive

To upload from your local drive, start with the following code:

In [2]:
from google.colab import files
uploaded = files.upload()

Saving carrfour_trans_pred.csv to carrfour_trans_pred.csv


It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).

In [0]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['carrfour_trans_pred.csv']),sep='|')
#df = pd.read_excel(io.BytesIO(uploaded['carrfour_trans_pred.xlsx']),encoding='ANSI')
#df = pd.read_excel(uploaded['carrfour_trans_pred.xlsx'],encoding='unicode')

In [4]:
# Dataset is now stored in a Pandas Dataframe
df[:2]

,Unnamed: 0,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by,text,sort_columns,labeled_by,cc3_labeled,cc4_labeled,cc5_labeled
0,0,fr,Desperados Sangré Bière Aromatisée Tequila Bou...,Drive Boissons Bières Aromatisees,unknown,Desperados Sangré Aromatisierte Bier Tequila F...,https://drive.carrefour.eu/fr/Bi%C3%A8res/Arom...,res aromatisees desperados sangr aromatis tequ...,paquet,21_Alcoholic beverages,213_Beer,2132_Other alcoholic beer,21_Alcoholic beverages,213_Beer,2134_Beer-based drinks,NaN,DESPERADOS,"5,69€",6143555.0,unknown,Desperados Sangré Bière Aromatisée Tequila Bou...,60,Lukas,NaN,NaN,NaN
1,1,fr,Hertekamp Belgian Oude Jenever 70 cl,Drive Boissons Apéritifs & alcools Genièvre,Hertekamp 35° est un genièvre de grain de qual...,Hertekamp Belgier Oude Jenever 70 cl,https://drive.carrefour.eu/fr/Ap%C3%A9ritifs-%...,ritifs alcools geni vre hertekamp belgian oude...,pièce,21_Alcoholic beverages,211_Spirits,2134_Beer-based drinks,21_Alcoholic beverages,211_Spirits,2134_Beer-based drinks,NaN,Hertekamp,"14,49€",5650237.0,Erik,Hertekamp Belgian Oude Jenever 70 cl Drive Boi...,60,NaN,NaN,NaN,NaN


## Upload from Google Drive

Loading CSV files stored in your google drive.

In [5]:
from google.colab import drive, files

drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


The commands will bring you to a Google Authentication step. You should see a screen with Google Drive File Stream wants to access your Google Account. After you allow permission, copy the given verification code and paste it in the box in Colab.
In the notebook, click on the charcoal > on the top left of the notebook and click on Files. Locate the data folder you created earlier and find your data. Right-click on your data and select Copy Path. Store this copied path into a variable and you are ready to go.

Specify the exact path to your file and the seperator of your file to read it in.

In [0]:
import pandas as pd

data_path = '/data/'#
file_path = 'fra/carrfour_trans_pred.csv'
df = pd.read_csv(path+data_path+file_path,sep='|',index_col=False)

Print out the dataframe to see if the import is correct.

In [7]:
df[:2]

,Unnamed: 0,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by,text,sort_columns,labeled_by,cc3_labeled,cc4_labeled,cc5_labeled
0,0,fr,Desperados Sangré Bière Aromatisée Tequila Bou...,Drive Boissons Bières Aromatisees,unknown,Desperados Sangré Aromatisierte Bier Tequila F...,https://drive.carrefour.eu/fr/Bi%C3%A8res/Arom...,res aromatisees desperados sangr aromatis tequ...,paquet,21_Alcoholic beverages,213_Beer,2132_Other alcoholic beer,21_Alcoholic beverages,213_Beer,2134_Beer-based drinks,NaN,DESPERADOS,"5,69€",6143555.0,unknown,Desperados Sangré Bière Aromatisée Tequila Bou...,60,Lukas,NaN,NaN,NaN
1,1,fr,Hertekamp Belgian Oude Jenever 70 cl,Drive Boissons Apéritifs & alcools Genièvre,Hertekamp 35° est un genièvre de grain de qual...,Hertekamp Belgier Oude Jenever 70 cl,https://drive.carrefour.eu/fr/Ap%C3%A9ritifs-%...,ritifs alcools geni vre hertekamp belgian oude...,pièce,21_Alcoholic beverages,211_Spirits,2134_Beer-based drinks,21_Alcoholic beverages,211_Spirits,2134_Beer-based drinks,NaN,Hertekamp,"14,49€",5650237.0,Erik,Hertekamp Belgian Oude Jenever 70 cl Drive Boi...,60,NaN,NaN,NaN,NaN


# Predict CoiCop Level 4 FOOD / BEVERAGE

In [0]:
from tqdm import tqdm
import re 
import pickle
import json
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# load model
lstm_model = tf.keras.models.load_model('de_fr_mod_cc5.h5')

with open('coicop_5_4.txt') as json_file:#
    coicop_5_4 = json.load(json_file)

with open('coicop_5_3.txt') as json_file:#
    coicop_5_3 = json.load(json_file)

class predictor:
    '''
    Takes the text and makes beautiful predictions for coicop categories
    '''
    def __init__(self,df,name_col,cat_col,url_col,lang,label_cat5,embedding_dim=300,label_dict3=coicop_5_3,label_dict4=coicop_5_4
                 ,model=lstm_model,batchsize=100):
        self.embedding_dim = embedding_dim
        self.label_cat5 = label_cat5
        self.df = df
        if url_col is not None:
            self.df['url_text'] = self.df[url_col].apply(lambda x: (self.parse_url(x)))

        if cat_col is not None and url_col is not None:
            self.df['text'] = self.df[name_col] + self.df[cat_col] + self.df['url_text']
            print('using name, category and words in url as input')
        elif cat_col is not None:
            self.df['text'] = self.df[name_col] + self.df[cat_col] 
            print('using name and category as input')
        elif url_col is not None:
            self.df['text'] = self.df[name_col] + self.df['url_text'] 
            print('using name and words in url as input')
        else:
            self.df['text'] = self.df[name_col]
            print('using only name as input')
        self.lang = lang
        if self.lang == 'fr':
            print('using french embeddings')
            self.emb = pickle.load(open('data/fr_slim_embed.p', "rb" ) )
        self.model = model
        self.label_dict3 = label_dict3
        self.label_dict4 = label_dict4
        self.seq_len = int(np.quantile(df['text'].apply(lambda x: len(x.split())),.95))
        print('95% quantile no. of words per row is',self.seq_len,'(trained on 39)')
        self.labels5 = list(self.label_dict3.keys())[:74]
        self.labels5.append('9999_Non-Food')
        self.labels5.sort()
        self.batch=batchsize
        self.total = len(self.df)
        self.df['cc3_pred'] = None
        self.df['cc4_pred'] = None
        self.df['cc5_pred'] = None

    def parse_url(self,url):
        url_list = str(url).split('/')[3:]
        url_str = ' '.join(w for w in url_list).lower()
        url_str = re.sub('[^a-zäöüàáâéèêßœ]+', ' ', url_str)
        url_str = ' '.join(w for w in url_str.split() if len(w)>2)
        return url_str
            
    def prepro(self,line):
        if isinstance(line,str):
            text_str = ' '.join(str(t) for t in line.split())
            text_str = text_str.lower()
            for a,b in rep_dict.items():
                text_str = text_str.replace(a,b)
            text_str = re.sub('[^a-zäöüàáâéèêßœ]+', ' ', text_str)
        else: 
            text_str = str(line)
            #print(line)
        return text_str

    rep_dict = {'.':' ',
                    ',': ' ',
                    '&': ' ',
                    '-': ' ',
                    '/': ' ',
                    '%': ' percent ',
                    '1': ' one ',
                    '2': ' two ',
                    '3': ' three ',
                    '4': ' four ',
                    '5': ' five ',
                    '6': ' six ',
                    '7': ' seven ',
                    '8': ' eigth ',
                    '9': ' nine ',
                    '0': ' zero ',
                    ' l ':' liter ',
                    ' ml ':' liter '
                    }

    def t2s(self,line):
        #tokens = []
        sen_embed = np.zeros((self.embedding_dim,self.seq_len))
        line = self.prepro(line)
        words = line.split()
        for w in range(0,self.seq_len):
            try: 
                emb = self.emb[words[w]]
            except:
                emb = np.zeros(self.embedding_dim)
            #tokens.append(tok)
            sen_embed[:,w] = emb
        sen_embed = np.swapaxes(sen_embed,0,1)
        return sen_embed #np.array(tokens)

    def emb_to_pred(self,embeded):
        y_pred5 = self.model.predict(embeded)
        y_pred5_arg = y_pred5.argmax(axis=1)
        y_pr_lab5 = [self.labels5[y] for y in y_pred5_arg]
        y_pr_lab3 = [self.label_dict3[cc5] for cc5 in y_pr_lab5]
        y_pr_lab4 = [self.label_dict4[cc5] for cc5 in y_pr_lab5]
        return y_pr_lab3,y_pr_lab4,y_pr_lab5

    def predict(self):
        resid = self.total % self.batch
        epochs = self.total // self.batch
        for i in tqdm(range(0,epochs)):
            fr_ = i*self.batch
            to_ = (i+1)*self.batch + (i+1==epochs) * resid
            text = self.df['text'][fr_:to_]
            #text_pre = [str(self.prepro(t)) for t in text]
            #print(text_pre)
            text_emb = np.array([self.t2s(t) for t in text])
            text_prd = self.emb_to_pred(text_emb)
            self.df['cc3_pred'][fr_:to_] = text_prd[0]
            self.df['cc4_pred'][fr_:to_] = text_prd[1]
            self.df['cc5_pred'][fr_:to_] = text_prd[2]
        
        if self.label_cat5 is not None:
            df_acc = self.df[self.df[label_cat5].isna()==False]
            print()
            print('number of observation (labeled / all):',len(df_acc),'/',len(self.df),'consistency ',accuracy_score(df_acc['cc5_pred'],df_acc['cc5']))
        
    def get_df(self):
        return self.df



In [0]:
import coicop_model

In [6]:
# init predictor
  
name_col = 'name'
category_col = 'categ'
url_col = 'url'
lang = 'fr'
label_cat5 = 'cc5'


CoiCop_Predictor = coicop_model.predictor(df
                                          , name_col
                                          , category_col
                                          , url_col
                                          , lang
                                          , label_cat5)

using name, category and words in url as input
using french embeddings
95% quantile no. of words per row is 34 (trained on 39)


In [0]:
#create predictions
CoiCop_Predictor.predict()

# Annotation

For the annotation tool you have to specify the naes of certain columns in your dataframe which the tool will use.  
* Labeled_by takes your name and stores it after every item you labeld.  
* Text column 1 is the name of the column for the first line of text which will be displayed to help you labeling the product, usually this is the product name.  
* Text column 2 is the name of the column for the second line of text which will be displayed to help you labeling the product, this could be the category or the translation.  
* The URL column is the name of the column where the url of the scraped product is located (if it is in the dataframe). The url might be helpfull if you are unsure about the product nd want to look up the whole page on the website. If you do not have the url in your data you can leave it empty.  
* CoiCop 5 prediction is the column where the prediction is stored. The prediction will appear preselected in the dropdown menu.   
  
The data are sorted in a way that catogies with few labels come first, if predictions of those label are available.


In [0]:
# init labeler

labeled_by = 'Erik'
text_column_1 = 'text'
text_column_2 = 'text_other'
url_column = 'url'
CoiCop_5_pred_col = 'cc5_pred'

CoiCop_Labeler = labeler_cc5.labeler( labeled_by
                        , df
                        , text_column_1
                        , text_column_2
                        , url_column
                        , CoiCop_5_pred_col)

Now you are ready to label, after executing the next line you will see:
* Text 1
* Text 2
* URL
* Dropdown with one category which is the prediction for the product. If you are dissatisfied with it, you can click on the dropdown and select the right one.
* Next-Button will skip this product
* Save-Button will annotate the selected category to your dataframe
  
If you finished labeling just jump to the next line.

In [9]:
CoiCop_Labeler.start_to_label()

The next line will print out your current progress of annotating the data.

In [10]:
CoiCop_Labeler.get_stats()

new labels: 4
in total 2915 of 8610 labeled ( 34.0 %)


Output the data including annotation.

In [13]:
df_new = CoiCop_Labeler.output_labels()
df_new[:5]

,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by,text,sort_columns,labeled_by,cc3_labeled,cc4_labeled,cc5_labeled
0,fr,Vandemoortele Délicieusement Sain 2 L,"Drive Epicerie Sauces, condiments & aides culi...",Avec 100% huile de tournesolRiche en acides gr...,Vandemoortele Köstlich gesund 2 L.,https://drive.carrefour.eu/fr/Sauces%2C-condim...,sauces condiments aides culinaires vinaigre hu...,pièce,NaN,NaN,NaN,11_Food,115_Oils and fats,1154_Other edible oils,NaN,Vandemoortele,"6,89€",6178214.0,unknown,Vandemoortele Délicieusement Sain 2 L Drive Ep...,61,NaN,NaN,NaN,NaN
1,fr,Vandemoortele Original 2 L,"Drive Epicerie Sauces, condiments & aides culi...",Toutes les qualités d'une bonne huile.Le meill...,Vandemoortele Original 2 L.,https://drive.carrefour.eu/fr/Sauces%2C-condim...,sauces condiments aides culinaires vinaigre hu...,pièce,NaN,NaN,NaN,11_Food,115_Oils and fats,1154_Other edible oils,NaN,Vandemoortele,"5,85€",5192607.0,unknown,Vandemoortele Original 2 L Drive Epicerie Sauc...,61,NaN,NaN,NaN,NaN
2,fr,Bertolli Originale 500 ml,"Drive Epicerie Sauces, condiments & aides culi...",Bertolli OriginaleHuile d'olive de catégorie s...,Bertolli Original 500 ml,https://drive.carrefour.eu/fr/Sauces%2C-condim...,sauces condiments aides culinaires vinaigre hu...,pièce,11_Food,115_Oils and fats,1153_Olive oil,11_Food,115_Oils and fats,1154_Other edible oils,NaN,Bertolli,"5,09€",1433607.0,unknown,Bertolli Originale 500 ml Drive Epicerie Sauce...,61,Erik,NaN,NaN,NaN
3,fr,Nutridia Bio Huile de Noix de Coco Extra Virgi...,Drive Bio & forme Epicerie Bio Epicerie salée ...,L'huile de noix de coco est facilement digesti...,Nutridia Bio Natives Kokosöl Extra 300 ml,https://drive.carrefour.eu/fr/Epicerie-Bio/Epi...,epicerie bio epicerie sal bio condiments sauce...,pièce,11_Food,115_Oils and fats,1154_Other edible oils,11_Food,119_Food products n.e.c.,1154_Other edible oils,NaN,Nutridia,"5,99€",5627697.0,unknown,Nutridia Bio Huile de Noix de Coco Extra Virgi...,61,Erik,NaN,NaN,NaN
4,fr,Vandemoortele Huile pour Salade 1 L,"Drive Epicerie Sauces, condiments & aides culi...",L'huile fait des merveilles.Notre Huile pour S...,Vandemoortele-Salatöl 1 l,https://drive.carrefour.eu/fr/Sauces%2C-condim...,sauces condiments aides culinaires vinaigre hu...,pièce,11_Food,115_Oils and fats,1154_Other edible oils,11_Food,115_Oils and fats,1154_Other edible oils,NaN,Vandemoortele,"3,19€",127859.0,Erik,Vandemoortele Huile pour Salade 1 L Drive Epic...,61,NaN,NaN,NaN,NaN


## Save data to Local Drive

In [0]:
from google.colab import files

df_new.to_csv('df_new.csv')
files.download('df_new.csv')

## Save data to Google Drive

In [0]:
if True:
    print(path+data_path+file_path,'saved')
    df_new.to_csv(path+data_path+file_path,sep='|')